In [ ]:
import fastf1
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


try:
    fastf1.Cache.enable_cache("f1_cache")
except Exception as e:
    print(f"Error enabling cache: {e}. Check cache directory permissions.")

def get_driver_features(driver_code):

    driver_characteristics = {
     
        'VER': {'WetWeatherSkill': 0.95, 'QualifyingPace': 0.95, 'RaceCraft': 0.95, 'Consistency': 0.90, 'Aggression': 0.90, 'TireManagement': 0.85},
        'NOR': {'WetWeatherSkill': 0.85, 'QualifyingPace': 0.92, 'RaceCraft': 0.88, 'Consistency': 0.88, 'Aggression': 0.75, 'TireManagement': 0.87},
        'PIA': {'WetWeatherSkill': 0.82, 'QualifyingPace': 0.88, 'RaceCraft': 0.84, 'Consistency': 0.85, 'Aggression': 0.78, 'TireManagement': 0.85},
        'LEC': {'WetWeatherSkill': 0.82, 'QualifyingPace': 0.95, 'RaceCraft': 0.85, 'Consistency': 0.80, 'Aggression': 0.85, 'TireManagement': 0.80},
        'HAM': {'WetWeatherSkill': 0.95, 'QualifyingPace': 0.90, 'RaceCraft': 0.95, 'Consistency': 0.92, 'Aggression': 0.80, 'TireManagement': 0.90},
        'RUS': {'WetWeatherSkill': 0.80, 'QualifyingPace': 0.90, 'RaceCraft': 0.85, 'Consistency': 0.85, 'Aggression': 0.83, 'TireManagement': 0.82},
        'ANT': {'WetWeatherSkill': 0.78, 'QualifyingPace': 0.85, 'RaceCraft': 0.78, 'Consistency': 0.76, 'Aggression': 0.85, 'TireManagement': 0.75},
        'SAI': {'WetWeatherSkill': 0.84, 'QualifyingPace': 0.88, 'RaceCraft': 0.87, 'Consistency': 0.87, 'Aggression': 0.82, 'TireManagement': 0.85},
        'ALO': {'WetWeatherSkill': 0.92, 'QualifyingPace': 0.90, 'RaceCraft': 0.95, 'Consistency': 0.90, 'Aggression': 0.88, 'TireManagement': 0.90},
        'STR': {'WetWeatherSkill': 0.85, 'QualifyingPace': 0.75, 'RaceCraft': 0.80, 'Consistency': 0.75, 'Aggression': 0.75, 'TireManagement': 0.78},
        'TSU': {'WetWeatherSkill': 0.78, 'QualifyingPace': 0.85, 'RaceCraft': 0.82, 'Consistency': 0.75, 'Aggression': 0.90, 'TireManagement': 0.78},
        'HAD': {'WetWeatherSkill': 0.75, 'QualifyingPace': 0.80, 'RaceCraft': 0.78, 'Consistency': 0.74, 'Aggression': 0.85, 'TireManagement': 0.74},
        'ALB': {'WetWeatherSkill': 0.80, 'QualifyingPace': 0.85, 'RaceCraft': 0.84, 'Consistency': 0.82, 'Aggression': 0.75, 'TireManagement': 0.83},
        'GAS': {'WetWeatherSkill': 0.83, 'QualifyingPace': 0.84, 'RaceCraft': 0.85, 'Consistency': 0.83, 'Aggression': 0.80, 'TireManagement': 0.82},
        'OCO': {'WetWeatherSkill': 0.80, 'QualifyingPace': 0.82, 'RaceCraft': 0.83, 'Consistency': 0.80, 'Aggression': 0.85, 'TireManagement': 0.78},
        'HUL': {'WetWeatherSkill': 0.85, 'QualifyingPace': 0.82, 'RaceCraft': 0.80, 'Consistency': 0.83, 'Aggression': 0.75, 'TireManagement': 0.80},
        'BOR': {'WetWeatherSkill': 0.74, 'QualifyingPace': 0.76, 'RaceCraft': 0.76, 'Consistency': 0.74, 'Aggression': 0.82, 'TireManagement': 0.75},
        'DOO': {'WetWeatherSkill': 0.74, 'QualifyingPace': 0.78, 'RaceCraft': 0.75, 'Consistency': 0.73, 'Aggression': 0.85, 'TireManagement': 0.74},
        'BEA': {'WetWeatherSkill': 0.75, 'QualifyingPace': 0.78, 'RaceCraft': 0.77, 'Consistency': 0.75, 'Aggression': 0.80, 'TireManagement': 0.76},
        'LAW': {'WetWeatherSkill': 0.76, 'QualifyingPace': 0.79, 'RaceCraft': 0.78, 'Consistency': 0.75, 'Aggression': 0.85, 'TireManagement': 0.75},
        
    }
    
  
    default_features = {
        'WetWeatherSkill': 0.75, 
        'QualifyingPace': 0.75, 
        'RaceCraft': 0.75, 
        'Consistency': 0.75, 
        'Aggression': 0.75, 
        'TireManagement': 0.75
    }
    
    # Return driver features if found, otherwise default
    if driver_code in driver_characteristics:
        return driver_characteristics[driver_code]
    # Try to handle numeric driver codes from historical data
    elif str(driver_code) in driver_characteristics:
        return driver_characteristics[str(driver_code)]
    else:
        print(f"Driver characteristics not found for '{driver_code}'. Using default values.")
        return default_features

def get_standings_before_round(year, target_round):

    driver_standings = {}
    constructor_standings = {}
    max_driver_points = 0
    max_constructor_points = 0
    try:
        schedule = fastf1.get_event_schedule(year, include_testing=False)
        races_before = schedule[schedule['RoundNumber'] < target_round]

        if races_before.empty:
            return {}, 0, {}, 0

        for index, race in races_before.iterrows():
            try:
                session = fastf1.get_session(year, race['EventName'], 'R')
                session.load(laps=False, telemetry=False, weather=False, messages=False)
                results = session.results
                if results is not None and not results.empty:
                    for _, driver_result in results.iterrows():
                        driver = driver_result['Abbreviation']
                        team = driver_result['TeamName']
                        points = driver_result['Points']
                        # Update driver standings
                        driver_standings[driver] = driver_standings.get(driver, 0) + points
                        # Update constructor standings
                        constructor_standings[team] = constructor_standings.get(team, 0) + points
            except Exception as e:
                # Silently continue if a past race fails to load
                continue

        if driver_standings:
            max_driver_points = max(driver_standings.values()) if driver_standings else 0
        if constructor_standings:
             max_constructor_points = max(constructor_standings.values()) if constructor_standings else 0

    except Exception as e:
        print(f"Error calculating standings for {year} before round {target_round}: {e}")
        return {}, 0, {}, 0

    return driver_standings, max_driver_points, constructor_standings, max_constructor_points

def get_race_data(year, event_identifier):
    try:
        session_r = fastf1.get_session(year, event_identifier, 'R')
        session_r.load(laps=False, telemetry=False, weather=False, messages=False)
        results_r = session_r.results

        if results_r is None or results_r.empty:
            print(f"No race results found for {year} {event_identifier}. Skipping.")
            return None

        results_r['FinishingPosition'] = pd.to_numeric(results_r['Position'], errors='coerce')
        results_r['GridPosition'] = pd.to_numeric(results_r['GridPosition'], errors='coerce')  # Add GridPosition
        results_r.dropna(subset=['FinishingPosition', 'GridPosition'], inplace=True)  # Ensure both positions are valid
        results_r[['FinishingPosition', 'GridPosition']] = results_r[['FinishingPosition', 'GridPosition']].astype(int)

        session_q = fastf1.get_session(year, event_identifier, 'Q')
        session_q.load(laps=False, telemetry=False, weather=False, messages=False)
        results_q = session_q.results

        if results_q is None or results_q.empty:
            print(f"No qualifying results found for {year} {event_identifier}. Proceeding without Quali times.")
            qualifying_times = pd.DataFrame({'Abbreviation': results_r['Abbreviation'], 'QualifyingTime (s)': np.nan})
        else:
            results_q['QualifyingTime'] = results_q[['Q1', 'Q2', 'Q3']].min(axis=1)
            results_q['QualifyingTime (s)'] = results_q['QualifyingTime'].dt.total_seconds()
            qualifying_times = results_q[['Abbreviation', 'QualifyingTime (s)']].copy()
        
        race_data = pd.merge(results_r[['Abbreviation', 'TeamName', 'FinishingPosition', 'GridPosition']], 
                             qualifying_times, on='Abbreviation', how='left')
        
        race_data['Year'] = year
        race_data['RoundNumber'] = session_r.event['RoundNumber']
        race_data['EventName'] = session_r.event['EventName']
        
        # Add driver characteristics
        for idx, row in race_data.iterrows():
            driver_code = row['Abbreviation']
            driver_features = get_driver_features(driver_code)
            for feature_name, feature_value in driver_features.items():
                race_data.at[idx, feature_name] = feature_value
        
        # Calculate Points Index *before* this race using the updated function
        driver_standings, max_driver_points, constructor_standings, max_constructor_points = get_standings_before_round(year, race_data['RoundNumber'].iloc[0])
        
        # Calculate Driver Points Index
        if max_driver_points > 0:
            race_data['PointsIndex'] = race_data['Abbreviation'].apply(lambda x: driver_standings.get(x, 0) / max_driver_points)
        else:
            race_data['PointsIndex'] = 0.0
            
        # Calculate Constructor Points Index
        if max_constructor_points > 0:
            race_data['ConstructorPointsIndex'] = race_data['TeamName'].apply(lambda x: constructor_standings.get(x, 0) / max_constructor_points)
        else:
             race_data['ConstructorPointsIndex'] = 0.0
             
        race_data.rename(columns={'Abbreviation': 'Driver', 'TeamName': 'Team'}, inplace=True)
        
        mean_quali_time = race_data['QualifyingTime (s)'].mean()
        race_data['QualifyingTime (s)'] = race_data['QualifyingTime (s)'].fillna(mean_quali_time)
        
        # Ensure no NaNs remain in core features
        core_features = ['Driver', 'Team', 'FinishingPosition', 'QualifyingTime (s)', 
                         'PointsIndex', 'ConstructorPointsIndex', 'GridPosition']
        driver_feature_keys = list(get_driver_features('VER').keys())  # Get a sample of driver feature keys
        core_features.extend(driver_feature_keys)
        race_data.dropna(subset=core_features, inplace=True)
        
        # Include driver features in the returned columns
        return_columns = ['Year', 'EventName', 'Driver', 'Team', 'QualifyingTime (s)', 
                          'PointsIndex', 'ConstructorPointsIndex', 'GridPosition', 'FinishingPosition']
        return_columns.extend(driver_feature_keys)
        
        return race_data[return_columns]
        
    except Exception as e:
        print(f"Error processing {year} {event_identifier}: {e}")
        return None

# --- Configuration ---
TARGET_YEAR = 2025
TARGET_GP_NAME = 'Saudi' 
HISTORICAL_YEARS = range(TARGET_YEAR - 1, 2017, -1) 


all_race_data = []

try:
    schedule_target_year = fastf1.get_event_schedule(TARGET_YEAR, include_testing=False)
    target_event = schedule_target_year[schedule_target_year['EventName'].str.contains(TARGET_GP_NAME, case=False, na=False)]
    
    if target_event.empty:
        raise ValueError(f"Target GP '{TARGET_GP_NAME}' not found in {TARGET_YEAR} schedule.")
        
    target_round = target_event['RoundNumber'].iloc[0]
    target_venue = target_event['Location'].iloc[0]
    target_event_name = target_event['EventName'].iloc[0] # Official name for consistency
    print(f"Target Event: {TARGET_YEAR} {target_event_name} (Round {target_round}, Venue: {target_venue})")

except Exception as e:
    raise ValueError(f"Error finding target GP info: {e}")

# 2. Get Data from Current Season (up to target GP)
print(f"\n--- Fetching {TARGET_YEAR} data before Round {target_round} ---")
schedule_current_year_filtered = schedule_target_year[schedule_target_year['RoundNumber'] < target_round]
for index, race in schedule_current_year_filtered.iterrows():
    print(f"Processing {TARGET_YEAR} {race['EventName']}...")
    data = get_race_data(TARGET_YEAR, race['EventName'])
    if data is not None:
        all_race_data.append(data)

# 3. Get Historical Data
print(f"\n--- Fetching Historical Data ---")
for year in HISTORICAL_YEARS:
    try:
        schedule_hist = fastf1.get_event_schedule(year, include_testing=False)
        print(f"Processing all races for {year}...")
        # Iterate through all races in the historical year's schedule
        for index, race_hist in schedule_hist.iterrows():
            event_name_hist = race_hist['EventName']
            print(f"  Processing {year} {event_name_hist}...")
            data = get_race_data(year, event_name_hist)
            if data is not None:
                all_race_data.append(data)
    except Exception as e:
        print(f"Error getting schedule or processing races for {year}: {e}")

# 4. Combine Data
if not all_race_data:
    raise ValueError("No data could be collected. Check FastF1 setup and GP/Year validity.")

combined_data = pd.concat(all_race_data, ignore_index=True)

# Drop the RoundNumber column
if 'RoundNumber' in combined_data.columns:
    combined_data.drop(columns=['RoundNumber'], inplace=True)
    print("Dropped 'RoundNumber' column.")
else:
    print("'RoundNumber' column not found in combined_data.")

print(f"\n--- Combined Data Shape after dropping RoundNumber: {combined_data.shape} ---")
print(combined_data.head())
print(combined_data.info())
print(combined_data.describe())

# Export the combined dataset to CSV
try:
    csv_filename = 'combined_f1_data.csv'
    combined_data.to_csv(csv_filename, index=False)
    print(f"\n--- Full Combined Dataset exported to {csv_filename} ---")
except Exception as e:
    print(f"\nError exporting data to CSV: {e}")

# 5. Prepare Data for Target Race Prediction
print(f"\n--- Preparing data for {TARGET_YEAR} {target_event_name} prediction ---")
try:
    # Get Qualifying data for the actual target race
    target_q_session = fastf1.get_session(TARGET_YEAR, target_event_name, 'Q')
    target_q_session.load(laps=False, telemetry=False, weather=False, messages=False)
    target_q_results = target_q_session.results
    
    if target_q_results is None or target_q_results.empty:
         raise ValueError(f"Could not load Qualifying results for target event: {TARGET_YEAR} {target_event_name}")
         
    target_q_results['QualifyingTime'] = target_q_results[['Q1', 'Q2', 'Q3']].min(axis=1)
    target_q_results['QualifyingTime (s)'] = target_q_results['QualifyingTime'].dt.total_seconds()
    
    # Get points index before the target race using the updated function
    target_driver_standings, target_max_driver_points, target_constructor_standings, target_max_constructor_points = get_standings_before_round(TARGET_YEAR, target_round)
    
    # Create DataFrame for prediction
    predict_df = target_q_results[['Abbreviation', 'TeamName', 'QualifyingTime (s)']].copy()
    predict_df.rename(columns={'Abbreviation': 'Driver', 'TeamName': 'Team'}, inplace=True)
    
    # Calculate Driver Points Index for prediction set
    if target_max_driver_points > 0:
         predict_df['PointsIndex'] = predict_df['Driver'].apply(lambda x: target_driver_standings.get(x, 0) / target_max_driver_points)
    else:
         predict_df['PointsIndex'] = 0.0
         
    # Calculate Constructor Points Index for prediction set
    if target_max_constructor_points > 0:
         predict_df['ConstructorPointsIndex'] = predict_df['Team'].apply(lambda x: target_constructor_standings.get(x, 0) / target_max_constructor_points)
    else:
         predict_df['ConstructorPointsIndex'] = 0.0
         
    # Handle potential missing Quali times in prediction set (e.g., pit lane start)
   
    mean_quali_train = combined_data['QualifyingTime (s)'].mean()
    predict_df['QualifyingTime (s)'] = predict_df['QualifyingTime (s)'].fillna(mean_quali_train)
    
    # Sort the qualifying results by time to determine grid positions
    predict_df = predict_df.sort_values('QualifyingTime (s)')
    predict_df['GridPosition'] = range(1, len(predict_df) + 1)
    
    # Add driver features to the prediction dataframe for the target race
    for idx, row in predict_df.iterrows():
        driver_code = row['Driver']
        driver_features = get_driver_features(driver_code)
        for feature_name, feature_value in driver_features.items():
            predict_df.at[idx, feature_name] = feature_value
    
    # Ensure no NaNs in prediction features
    predict_df.dropna(subset=['QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex'], inplace=True)
    
    # Keep track of drivers for final output
    predict_drivers = predict_df['Driver'].tolist()
    
    print(f"Prediction input data shape: {predict_df.shape}")
    print(predict_df)
    
except Exception as e:
    raise ValueError(f"Error preparing prediction data: {e}")

In [17]:
features = ['GridPosition', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex','QualifyingPace', 'RaceCraft', 'Consistency', 'Aggression', 'TireManagement']
target = 'FinishingPosition'

X = combined_data[features].copy()
y = combined_data[target].copy()

predict_X = predict_df[features].copy()


print(predict_X.head())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(
    n_estimators=100, 
    learning_rate=0.2, 
    max_depth=3, 
    random_state=42
    )

model.fit(X_train, y_train)

predicted_scores = model.predict(predict_X)

#Create Ranked Results

results_df = pd.DataFrame({
    'Driver': predict_drivers, 
    'PredictedScore': predicted_scores
})

results_df = results_df.sort_values(by='PredictedScore')
# Assign ranks 
results_df['PredictedFinishingPosition'] = np.arange(1, len(results_df) + 1)


print(f"\Predicted Finishing Order for {TARGET_YEAR} {target_event_name} \n")
print(results_df[['Driver', 'PredictedFinishingPosition']].to_string(index=False))


print("\n--- Evaluating Model Performance ---")
y_pred_test = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f"Mean Absolute Error on test set is {mae:.2f} positions")

print("\nFeature Importance")
try:
    feature_names = X.columns
    importances = model.feature_importances_
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    print(importance_df.to_string(index=False))
except Exception as e:
    print(f"Could not calculate or display feature importances: {e}")

    GridPosition  QualifyingTime (s)  PointsIndex  ConstructorPointsIndex  \
1              1              87.294     0.828947                0.454545   
81             2              87.304     0.881579                1.000000   
63             3              87.407     0.763158                0.601399   
4              4              87.481     1.000000                1.000000   
16             5              87.670     0.368421                0.314685   

    QualifyingPace  RaceCraft  Consistency  Aggression  TireManagement  
1             0.95       0.95         0.90        0.90            0.85  
81            0.88       0.84         0.85        0.78            0.85  
63            0.90       0.85         0.85        0.83            0.82  
4             0.92       0.88         0.88        0.75            0.87  
16            0.95       0.85         0.80        0.85            0.80  
\Predicted Finishing Order for 2025 Saudi Arabian Grand Prix 

Driver  PredictedFinishingPosition
 

In [18]:
# Cell for Random Forest Regressor Implementation
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
# Assuming train_test_split and mean_absolute_error are already imported
# Assuming combined_data, predict_df, predict_drivers, X_train, X_test, y_train, y_test are available

print("\\n--- Preparing Data for Random Forest Regressor ---")

# Use the same features and target as before
features_rf = ['GridPosition', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex',
               'QualifyingPace', 'RaceCraft', 'Consistency', 'Aggression', 'TireManagement']
target_rf = 'FinishingPosition'

# use training and test set from gradient boosting 

# --- Model Training ---
print("\\n--- Training Random Forest Regressor ---")

rf_regressor = RandomForestRegressor(
    n_estimators=100,      
    max_depth=None,         
    min_samples_split=2,    
    min_samples_leaf=1,    
    random_state=42,
    n_jobs=-1              
)

rf_regressor.fit(X_train, y_train) 

# --- Prediction ---
print(f"\\n--- Predicting with Random Forest Regressor for {TARGET_YEAR} {target_event_name} ---")


predict_X_rf = predict_df[features_rf].copy() 

predicted_rf_scores = rf_regressor.predict(predict_X_rf)

# Create Ranked Results
results_rf_df = pd.DataFrame({
    'Driver': predict_drivers, 
    'PredictedRFScore': predicted_rf_scores 
})

results_rf_df = results_rf_df.sort_values(by='PredictedRFScore')
results_rf_df['PredictedFinishingPosition'] = np.arange(1, len(results_rf_df) + 1)

print(f"\\nPredicted Finishing Order (Random Forest) for {TARGET_YEAR} {target_event_name}\\n")
print(results_rf_df[['Driver', 'PredictedFinishingPosition']].to_string(index=False))

# --- Evaluation ---
print("\\n--- Evaluating Random Forest Model Performance ---")
y_pred_test_rf = rf_regressor.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_test_rf) 
print(f"Mean Absolute Error on test set is {mae_rf:.2f} positions")

# --- Feature Importance ---
print("\\nFeature Importance (Random Forest)")
try:
    feature_names_rf = X_train.columns
    importances_rf = rf_regressor.feature_importances_
    importance_rf_df = pd.DataFrame({'Feature': feature_names_rf, 'Importance': importances_rf})
    importance_rf_df = importance_rf_df.sort_values(by='Importance', ascending=False)
    print(importance_rf_df.to_string(index=False))
except Exception as e:
    print(f"Could not calculate or display feature importances: {e}")


\n--- Preparing Data for Random Forest Regressor ---
\n--- Training Random Forest Regressor ---
\n--- Predicting with Random Forest Regressor for 2025 Saudi Arabian Grand Prix ---
\nPredicted Finishing Order (Random Forest) for 2025 Saudi Arabian Grand Prix\n
Driver  PredictedFinishingPosition
   PIA                           1
   NOR                           2
   RUS                           3
   VER                           4
   LEC                           5
   ANT                           6
   HAM                           7
   GAS                           8
   TSU                           9
   SAI                          10
   ALO                          11
   ALB                          12
   HAD                          13
   BEA                          14
   LAW                          15
   OCO                          16
   DOO                          17
   BOR                          18
   STR                          19
   HUL                          20
\n---

In [21]:
# Cell for LambdaMART (LightGBM Ranker) Implementation
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score

print("\\n--- Preparing Data for LambdaMART (LightGBM Ranker) ---")


features_ltr = ['Driver', 'GridPosition', 'QualifyingTime (s)', 'PointsIndex', 'ConstructorPointsIndex']
target_ltr = 'RelevanceScore'

# --- Data Preparation for LTR ---
combined_data_ltr = combined_data.copy()



combined_data_ltr['Driver'] = combined_data_ltr['Driver'].astype('category')

combined_data_ltr[target_ltr] = 21 - combined_data_ltr['FinishingPosition']
combined_data_ltr['QueryID'] = combined_data_ltr.groupby(['Year', 'EventName']).ngroup()

X_ltr_full = combined_data_ltr[features_ltr]
y_ltr_full = combined_data_ltr[target_ltr]
groups_ltr_full = combined_data_ltr['QueryID']

# --- Train-Test Split for LTR ---
print("\\n--- Performing Train-Test Split for LambdaMART ---")
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(gss.split(X_ltr_full, y_ltr_full, groups_ltr_full))

X_train_ltr, X_test_ltr = X_ltr_full.iloc[train_idx], X_ltr_full.iloc[test_idx]
y_train_ltr, y_test_ltr = y_ltr_full.iloc[train_idx], y_ltr_full.iloc[test_idx]

groups_train_ltr_counts = combined_data_ltr.iloc[train_idx].groupby('QueryID').size().tolist()
query_ids_test_ltr = combined_data_ltr.iloc[test_idx]['QueryID']
groups_test_ltr_counts = combined_data_ltr.iloc[test_idx].groupby('QueryID').size().tolist()

# --- Model Training ---
print("\\n--- Training LambdaMART (LightGBM Ranker) ---")

lgbm_ranker = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10, 20],
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42,
    # No need for categorical_feature in fit if dtypes are 'category'
)

if not X_train_ltr.empty and not X_test_ltr.empty and groups_train_ltr_counts and groups_test_ltr_counts:
    lgbm_ranker.fit(X_train_ltr, y_train_ltr, group=groups_train_ltr_counts,
                    eval_set=[(X_test_ltr, y_test_ltr)],
                    eval_group=[groups_test_ltr_counts], # Corrected to pass the list of group sizes
                    eval_metric='ndcg',
                    eval_at=[1, 3, 5, 10, 20],
                    callbacks=[lgb.early_stopping(10, verbose=True)])
else:
    print("Training or Test set for LTR is empty or group counts are missing. Skipping LTR model training and evaluation.")

# --- Prediction on Target Event ---
if hasattr(lgbm_ranker, 'booster_') and lgbm_ranker.booster_: # Check if model was trained
    print(f"\\n--- Predicting with LambdaMART for {TARGET_YEAR} {target_event_name} ---")

    # Ensure 'Driver' in predict_df is also category and has same categories as training data
    predict_df_ltr = predict_df.copy()
    predict_df_ltr['Driver'] = pd.Categorical(predict_df_ltr['Driver'], categories=X_train_ltr['Driver'].cat.categories)
    predict_X_ltr = predict_df_ltr[features_ltr]

   

    predicted_ltr_scores_target_event = lgbm_ranker.predict(predict_X_ltr)

    results_ltr_df_target_event = pd.DataFrame({
        'Driver': predict_drivers, # predict_drivers should be available from your previous cells
        'PredictedLTRScore': predicted_ltr_scores_target_event
    })
    results_ltr_df_target_event = results_ltr_df_target_event.sort_values(by='PredictedLTRScore', ascending=False)
    results_ltr_df_target_event['PredictedFinishingPosition'] = np.arange(1, len(results_ltr_df_target_event) + 1)

    print(f"\\nPredicted Finishing Order (LambdaMART) for {TARGET_YEAR} {target_event_name}\\n")
    print(results_ltr_df_target_event[['Driver', 'PredictedFinishingPosition']].to_string(index=False))

    # --- Evaluation on Test Set ---
    print("\\n--- Evaluating LambdaMART Model Performance on Test Set ---")
    if not X_test_ltr.empty:
        test_preds_ltr = lgbm_ranker.predict(X_test_ltr)
        ndcg_scores_per_query = []
        k_val_overall = 20
        for query_id in sorted(query_ids_test_ltr.unique()):
            query_mask = (query_ids_test_ltr == query_id)
            true_relevance_for_query = y_test_ltr[query_mask].values.reshape(1, -1)
            predicted_scores_for_query = test_preds_ltr[query_mask].reshape(1, -1)

            if true_relevance_for_query.shape[1] > 1:
                k_val = min(k_val_overall, true_relevance_for_query.shape[1])
                if k_val > 0:
                    ndcg_val = ndcg_score(true_relevance_for_query, predicted_scores_for_query, k=k_val)
                    ndcg_scores_per_query.append(ndcg_val)

        if ndcg_scores_per_query:
            mean_ndcg = np.mean(ndcg_scores_per_query)
            print(f"Mean NDCG@{k_val_overall} (Ranking Accuracy Percentage) on test set: {mean_ndcg*100:.2f}%") # Using the modified print
        else:
            print("Could not calculate Mean NDCG. Test set might be too small or group distribution issues.")
    else:
        print("Test set for LTR is empty. Skipping NDCG evaluation.")

    print("\\nFeature Importance (LambdaMART)")
    try:
        feature_names_ltr = X_train_ltr.columns
        importances_ltr = lgbm_ranker.feature_importances_
        importance_ltr_df = pd.DataFrame({'Feature': feature_names_ltr, 'Importance': importances_ltr})
        importance_ltr_df = importance_ltr_df.sort_values(by='Importance', ascending=False)
        print(importance_ltr_df.to_string(index=False))
    except Exception as e:
            print(f"Could not calculate or display feature importances: {e}")
else:
    print("LGBMRanker model was not trained (likely due to empty train/test sets or other issues). Skipping target prediction and evaluation.")


\n--- Preparing Data for LambdaMART (LightGBM Ranker) ---
\n--- Performing Train-Test Split for LambdaMART ---
\n--- Training LambdaMART (LightGBM Ranker) ---
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000105 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 823
[LightGBM] [Info] Number of data points in the train set: 1759, number of used features: 5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

e:\Projects\AI Project\.venv\Lib\site-packages\lightgbm\sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
e:\Projects\AI Project\.venv\Lib\site-packages\lightgbm\sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
e:\Projects\AI Project\.venv\Lib\site-packages\lightgbm\sklearn.py:861: UserWarning: Found 'ndcg_eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
